In [515]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/semantic-similarity/train - train.csv
/kaggle/input/semantic-similarity/evaluation - evaluation.csv


In [516]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [517]:
# Load pre-trained DistillBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [518]:
ideal="Artificial Intelligence (AI) refers to computer systems designed to perform tasks that typically require human intelligence. It encompasses various technologies like machine learning and deep learning, enabling machines to analyze data, recognize patterns, and make decisions. AI systems can excel in tasks such as image and speech recognition, natural language processing, and problem-solving. As an interdisciplinary field, AI aims to create machines capable of learning and adapting, advancing technology's ability to automate processes, enhance efficiency, and address complex challenges across industries, from healthcare and finance to transportation and beyond, heralding a transformative era in the way we interact with and utilize technology."

In [519]:
wrong="Birds, a diverse class of warm-blooded vertebrates, encompass a vast array of species adapted to environments worldwide. Renowned for their ability to fly, birds exhibit remarkable diversity in size, color, and behavior. From the majestic eagles soaring in the sky to the tiny hummingbirds darting among flowers, they play crucial ecological roles as pollinators, seed dispersers, and insect controllers. Birds, with their unique songs and intricate plumage, captivate humans, inspiring art, folklore, and scientific inquiry. Their migratory journeys across continents showcase astonishing feats of navigation. Birds serve as vital indicators of environmental health, emphasizing the interconnectedness of ecosystems."

In [520]:
student="Artificial Intelligence (AI) is a branch of computer science that empowers machines to simulate human intelligence. Utilizing algorithms, AI enables systems to learn from data, recognize patterns, and make decisions, mimicking cognitive functions. Applications span from natural language processing and image recognition to autonomous vehicles. AI is a transformative force, impacting industries, improving efficiency, and raising ethical considerations regarding privacy and bias. As technology evolves, AI continues to shape our daily lives, advancing fields like healthcare, finance, and robotics, while fostering a future where machines seamlessly interact with and augment human capabilities."

In [521]:
# Tokenize input text
tokens1 = tokenizer(ideal, return_tensors='pt')
tokens2 = tokenizer(student, return_tensors='pt')

In [522]:
# Generate embeddings
with torch.no_grad():
    embeddings1 = model(**tokens1).last_hidden_state.mean(dim=1)
    embeddings2 = model(**tokens2).last_hidden_state.mean(dim=1)

In [523]:
# Generate embeddings
with torch.no_grad():
    embeddings1 = model(**tokens1).last_hidden_state.mean(dim=1)
    embeddings2 = model(**tokens2).last_hidden_state.mean(dim=1)

# Calculate cosine similarity
similarity = cosine_similarity(embeddings1, embeddings2)
# print(f"Similarity: {similarity[0][0]}")
if similarity[0][0]>0.95:
    print("Two corpus are similar")
else:
    print("The corpus are not similar")

Two corpus are similar


In [524]:
# Tokenize input text
tokens1 = tokenizer(ideal, return_tensors='pt')
tokens2 = tokenizer(wrong, return_tensors='pt')

# Generate embeddings
with torch.no_grad():
    embeddings1 = model(**tokens1).last_hidden_state.mean(dim=1)
    embeddings2 = model(**tokens2).last_hidden_state.mean(dim=1)


# Generate embeddings
with torch.no_grad():
    embeddings1 = model(**tokens1).last_hidden_state.mean(dim=1)
    embeddings2 = model(**tokens2).last_hidden_state.mean(dim=1)

# Calculate cosine similarity
similarity = cosine_similarity(embeddings1, embeddings2)
if similarity[0][0]>0.90:
    print("Two corpus are similar")
else:
    print("The corpus are not similar")

The corpus are not similar


In [525]:
mid="Machine Learning (ML) is a subset of artificial intelligence that focuses on enabling computers to learn and improve from experience without being explicitly programmed."

In [526]:
# Tokenize input text
tokens1 = tokenizer(ideal, return_tensors='pt')
tokens2 = tokenizer(mid, return_tensors='pt')

# Generate embeddings
with torch.no_grad():
    embeddings1 = model(**tokens1).last_hidden_state.mean(dim=1)
    embeddings2 = model(**tokens2).last_hidden_state.mean(dim=1)


# Generate embeddings
with torch.no_grad():
    embeddings1 = model(**tokens1).last_hidden_state.mean(dim=1)
    embeddings2 = model(**tokens2).last_hidden_state.mean(dim=1)

# Calculate cosine similarity
similarity = cosine_similarity(embeddings1, embeddings2)

if similarity[0][0]>0.95:
    print("Two corpus are similar")
elif 0.95>similarity[0][0]>0.90:
    print("Middle similarity")
else:
    print("The corpus are not similar")

Middle similarity


*take up data*

In [527]:
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

In [528]:
df = pd.read_csv('/kaggle/input/semantic-similarity/train - train.csv')

In [529]:
df.head

<bound method NDFrame.head of       Unnamed: 0                                               text  \
0              0                  can not share audio via bluetooth   
1              1                      Google sign-in does not work!   
2              2     the best platform to communicate in a pandemic   
3              3                  it will not open on my telephone!   
4              4  this app is very expensive to learn study onli...   
...          ...                                                ...   
4117        4117  it does not allow me to change the language of...   
4118        4118                 the best app in populate cyclosis!   
4119        4119                    getting too many notifications.   
4120        4120  why can not you hear me without headphones dur...   
4121        4121       there is not touchup option available as yet   

                                     reason  label  
0             good for on-line presentation      0  
1          

In [530]:
tokens1 = tokenizer(df['text'].tolist(), return_tensors='pt', padding=True, truncation=True)
tokens2 = tokenizer(df['reason'].tolist(), return_tensors='pt', padding=True, truncation=True)

# Convert labels to tensor
labels = torch.tensor(df['label'].values).view(-1, 1)  # No need for float() as labels are already 0 or 1



In [531]:
class SimilarityModel(nn.Module):
    def __init__(self, hidden_size=768):
        super(SimilarityModel, self).__init__()
        self.distilbert = model
        self.fc = nn.Linear(2 * hidden_size, 1)  # Adjust input size based on your model

    def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
        outputs1 = self.distilbert(input_ids=input_ids1, attention_mask=attention_mask1)
        outputs2 = self.distilbert(input_ids=input_ids2, attention_mask=attention_mask2)
        last_hidden_state1 = outputs1.last_hidden_state.mean(dim=1)
        last_hidden_state2 = outputs2.last_hidden_state.mean(dim=1)
        concatenated = torch.cat((last_hidden_state1, last_hidden_state2), dim=1)
        output = self.fc(concatenated)
        return output


In [532]:
model = SimilarityModel(hidden_size=768)
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=1e-5)

In [533]:
dataset = TensorDataset(tokens1['input_ids'], tokens1['attention_mask'], tokens2['input_ids'], tokens2['attention_mask'], labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for batch in dataloader:
        optimizer.zero_grad()
        
        outputs = model(input_ids1=batch[0], attention_mask1=batch[1], input_ids2=batch[2], attention_mask2=batch[3])
        loss = criterion(outputs, batch[4].float())  # Convert labels to float
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

KeyboardInterrupt: 

In [ ]:
def get_similarity_score(model, tokenizer, text1, text2):
    # Tokenize input text
    tokens1 = tokenizer(text1, return_tensors='pt', padding=True, truncation=True)
    tokens2 = tokenizer(text2, return_tensors='pt', padding=True, truncation=True)

    # Model prediction
    with torch.no_grad():
        output1 = model(**tokens1)
        output2 = model(**tokens2)

        # Assuming the last hidden states are used for classification
        last_hidden_state1 = output1.last_hidden_state.mean(dim=1)
        last_hidden_state2 = output2.last_hidden_state.mean(dim=1)

        # Concatenate the embeddings
        concatenated = torch.cat((last_hidden_state1, last_hidden_state2), dim=1)

        # Pass through the linear layer
        similarity_score = torch.sigmoid(model.fc(concatenated)).item()

    return similarity_score

# Example usage
text1 = "This is the first statement."
text2 = "This is a similar statement."

similarity_score = get_similarity_score(model, tokenizer, text1, text2)
print(f"Similarity Score: {similarity_score}")


In [ ]:
ideal="Artificial Intelligence (AI) refers to computer systems designed to perform tasks that typically require human intelligence. It encompasses various technologies like machine learning and deep learning, enabling machines to analyze data, recognize patterns, and make decisions. AI systems can excel in tasks such as image and speech recognition, natural language processing, and problem-solving. As an interdisciplinary field, AI aims to create machines capable of learning and adapting, advancing technology's ability to automate processes, enhance efficiency, and address complex challenges across industries, from healthcare and finance to transportation and beyond, heralding a transformative era in the way we interact with and utilize technology."

ans="Birds, a diverse class of warm-blooded vertebrates, encompass a vast array of species adapted to environments worldwide. Renowned for their ability to fly, birds exhibit remarkable diversity in size, color, and behavior. From the majestic eagles soaring in the sky to the tiny hummingbirds darting among flowers, they play crucial ecological roles as pollinators, seed dispersers, and insect controllers. Birds, with their unique songs and intricate plumage, captivate humans, inspiring art, folklore, and scientific inquiry. Their migratory journeys across continents showcase astonishing feats of navigation. Birds serve as vital indicators of environmental health, emphasizing the interconnectedness of ecosystems."

# Tokenize input text
tokens1 = tokenizer(ideal, return_tensors='pt')
tokens2 = tokenizer(ans, return_tensors='pt')

# Generate embeddings
with torch.no_grad():
    embeddings1 = model(**tokens1).last_hidden_state.mean(dim=1)
    embeddings2 = model(**tokens2).last_hidden_state.mean(dim=1)

# Calculate cosine similarity
similarity = cosine_similarity(embeddings1, embeddings2)
print(f"Similarity: {similarity[0][0]}")